In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [5]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [8]:
# MNIST dataset
mnist_train = dsets.MNIST(root = 'MNIST_data/', train = True, transform = transforms.ToTensor(), download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/', train = False, transform = transforms.ToTensor(), download = True)



In [9]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size = batch_size, shuffle = True, drop_last= True)

In [14]:
# nn layers
linear1 = torch.nn.Linear(784,256, bias = True)
linear2 = torch.nn.Linear(256,256, bias = True)
linear3 = torch.nn.Linear(256,10, bias = True)
relu = torch.nn.ReLU()

In [15]:
# xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[ 0.0453,  0.1376,  0.0451,  ..., -0.0096,  0.1334, -0.1393],
        [-0.1257, -0.0956, -0.1335,  ..., -0.1085,  0.1030,  0.0254],
        [ 0.0200, -0.0099,  0.1434,  ...,  0.0372,  0.0269,  0.0924],
        ...,
        [-0.0163,  0.0534, -0.0541,  ...,  0.0991, -0.0501,  0.0980],
        [ 0.1128,  0.1445,  0.1208,  ..., -0.0029, -0.0115, -0.0077],
        [-0.1209, -0.0056, -0.0899,  ...,  0.0567,  0.0614, -0.0351]],
       requires_grad=True)

In [16]:
# model
model = torch.nn.Sequential(linear1, relu, linear2, relu,linear3).to(device)

In [17]:
# define cost/loss & optimizer

criterion = torch.nn.CrossEntropyLoss().to(device) #softmax is internally computed
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


In [18]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    # reshape input image into [batch_size by 784]
    # label is not one-hot encoded

    X = X.view(-1,28*28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost/total_batch

  print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.247675881
Epoch: 0002 cost = 0.093973473
Epoch: 0003 cost = 0.061888341
Epoch: 0004 cost = 0.044724789
Epoch: 0005 cost = 0.031902771
Epoch: 0006 cost = 0.024752134
Epoch: 0007 cost = 0.022567596
Epoch: 0008 cost = 0.016049428
Epoch: 0009 cost = 0.017939884
Epoch: 0010 cost = 0.015476191
Epoch: 0011 cost = 0.011830999
Epoch: 0012 cost = 0.009511827
Epoch: 0013 cost = 0.013351304
Epoch: 0014 cost = 0.009573714
Epoch: 0015 cost = 0.011500275
Learning finished


In [19]:
# Test the model useing test sets

with torch.no_grad():
  X_test = mnist_test.test_data.view(-1,28*28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1)== Y_test
  accuracy= correct_prediction.float().mean()
  print('Accuracy: ', accuracy.item() )

  # Get one and predict
  r= random.randint(0, len(mnist_test) - 1)
  X_single_data = mnist_test.test_data[r:r+1].view(-1, 28*28).float().to(device)
  Y_single_data = mnist_test.test_labels[r:r+1].to(device)

  print('Label: ', Y_single_data.item())
  single_prediction = model(X_single_data)
  print('Prediction: ', torch.argmax(single_prediction,1).item())


Accuracy:  0.9793000221252441
Label:  8
Prediction:  8


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
